In [1]:
df = spark.read.format("parquet")\
    .option("header","true")\
    .option("inferschema","true")\
    .load("s3://project.group23.fan5.flight/Scaled_Data_M/")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1648364081223_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.functions import col
trainDF = df.where(col("YEAR")!=2018)
testDF = df.where(col("YEAR")==2018)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
trainDF, testDF1 =  trainDF.randomSplit([0.25,0.75], seed = 143)
trainDF1, testDF =  testDF.randomSplit([0.75,0.25], seed = 143)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.ml.classification import GBTClassifier
gb = GBTClassifier(labelCol = 'FLIGHT_STATUS', featuresCol = 'scaledfeatures', maxIter=3, stepSize=1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
gbModel = gb.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:

gb_predictions = gbModel.transform(testDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'accuracy')
print('Accuracy:', multi_evaluator.evaluate(gb_predictions))
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'f1')
print('F1_Score:', multi_evaluator.evaluate(gb_predictions))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6647998524674605
F1_Score: 0.6126945311988082

In [10]:
from pyspark.ml.classification import GBTClassifier
gb2 = GBTClassifier(labelCol = 'FLIGHT_STATUS', featuresCol = 'scaledfeatures', maxIter=3, stepSize=0.1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
gbModel2 = gb2.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
gb_predictions2 = gbModel2.transform(testDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'accuracy')
print('Accuracy:', multi_evaluator.evaluate(gb_predictions2))
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'f1')
print('F1_Score:', multi_evaluator.evaluate(gb_predictions2))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6524705694487526
F1_Score: 0.5580815380457746

In [14]:
from pyspark.ml.classification import GBTClassifier
gb3 = GBTClassifier(labelCol = 'FLIGHT_STATUS', featuresCol = 'scaledfeatures', maxIter=20, stepSize=0.1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
gbModel3 = gb3.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
gb_predictions3 = gbModel3.transform(testDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'accuracy')
print('Accuracy:', multi_evaluator.evaluate(gb_predictions3))
multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'FLIGHT_STATUS', metricName = 'f1')
print('F1_Score:', multi_evaluator.evaluate(gb_predictions3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.6738947247931683
F1_Score: 0.6045922058700368

In [4]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
gb = GBTClassifier(labelCol = 'FLIGHT_STATUS', featuresCol = 'scaledfeatures')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
gbevaluator = MulticlassClassificationEvaluator(labelCol='FLIGHT_STATUS', predictionCol='prediction', metricName='accuracy')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
gbparamGrid = (ParamGridBuilder()
               .addGrid(gb.maxDepth, [3, 5, 7])
               .addGrid(gb.maxBins, [3, 4, 5])
               .addGrid(gb.maxIter, [5, 10, 20])
               .build())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
gbcv = CrossValidator(estimator = gb,
                      estimatorParamMaps = gbparamGrid,
                      evaluator = gbevaluator,
                      numFolds = 5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
gbcvModel = gbcv.fit(trainDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://ip-172-31-79-98.ec2.internal:8998/sessions/0/statements/9 with error payload: {"msg":"requirement failed: Session isn't active."}


In [11]:
writeStream.start()

An error was encountered:
Invalid status code '404' from http://ip-172-31-79-98.ec2.internal:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [12]:
spark.streams.awaitAnyTermination()

An error was encountered:
Invalid status code '404' from http://ip-172-31-79-98.ec2.internal:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [10]:
print(gbcvModel)

An error was encountered:
Invalid status code '404' from http://ip-172-31-79-98.ec2.internal:8998/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [ ]:
gbpredictions = gbcvModel.transform(testDF)

In [ ]:
print('RMSE:', gbevaluator.evaluate(gbpredictions))